Задача - суммаризация текста научных статей

Данные - оказывается данные с ArXiv есть в TensorFlow

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [2]:
builder  = tfds.builder('scientific_papers/arxiv')

In [3]:
# Скачать если не скачан предварительно (примерно 4 Гб(архив) + 15Гб окончательный размер на диске)
# Возможна проблема при экстракции данных из-за недостатка оперативки. 12.7Гб на колабе не хватило
# Я использовал факультетский кластер на 1Тб оперативки, в максимуме что я заметил, он требовал > 24Гб
builder.download_and_prepare()
# Однако даже после скачивания builder подготавливает данные с ошибкой, 
# вроде дело в парсере json ,но на дебаг просто нет времени


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

JSONDecodeError: Unterminated string starting at: line 1 column 89060 (char 89059)

# Начало тутъ

In [5]:
# Попытка номер 2 - HuggingFace 

In [1]:
from datasets import load_dataset

dataset = load_dataset("scientific_papers", 'arxiv')

Found cached dataset scientific_papers (C:/Users/v.shirobokov/.cache/huggingface/datasets/scientific_papers/arxiv/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


  0%|          | 0/3 [00:00<?, ?it/s]

Тут датасет удобно разбит на выборки. Сами выборки сразу поделены на абстракт и статью. Section names в данной задаче суммаризации я пока не вижу смысла использовать. 

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6440
    })
})

In [4]:
test_dataset = dataset['test']

Normalization

In [3]:
import torch
import spacy
from torch import nn
from torch.nn import functional as F

from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data.dataset import random_split

from torchtext.data.utils import get_tokenizer


Взял один экземпляр чтобы отладить часть кода с отчисткой и с токенизацией

In [7]:
test_token = test_dataset[0].copy()
test_token['section_names']

'introduction\nmethods of periodicity analysis\na method of the diagnosis of an echo-effect in the power spectrum\ndata analysis\nthe periodicity at about 155 days during the maximum activity period\nconclusion\nacknowledgments'

Проверял как быстро работает spacy на одном абстракте 

In [8]:
nlp = spacy.load("en_core_web_sm")

In [9]:
%%time
test_test_data = map(
    lambda x: ' '.join(
        token.lemma_.lower() for token in nlp(x) if 
        not token.is_stop 
        and not token.is_punct
        and not token.is_digit
        and not token.like_email
        and not token.like_num
        and not token.is_space
    ), test_token['abstract']
)
list(test_test_data)

CPU times: total: 6.78 s
Wall time: 6.8 s


['',
 't',
 'h',
 'e',
 '',
 's',
 'h',
 'o',
 'r',
 't',
 '',
 '',
 '',
 't',
 'e',
 'r',
 'm',
 '',
 'p',
 'e',
 'r',
 '',
 'o',
 'd',
 '',
 'c',
 '',
 't',
 '',
 'e',
 's',
 '',
 'o',
 'f',
 '',
 't',
 'h',
 'e',
 '',
 'd',
 '',
 '',
 'l',
 'y',
 '',
 's',
 'u',
 'n',
 's',
 'p',
 'o',
 't',
 '',
 '',
 'r',
 'e',
 '',
 '',
 'f',
 'l',
 'u',
 'c',
 't',
 'u',
 '',
 't',
 '',
 'o',
 'n',
 's',
 '',
 'f',
 'r',
 'o',
 'm',
 '',
 '',
 'u',
 'g',
 'u',
 's',
 't',
 '',
 '',
 '',
 '',
 '',
 '',
 't',
 'o',
 '',
 'o',
 'c',
 't',
 'o',
 'b',
 'e',
 'r',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'r',
 'e',
 '',
 'd',
 '',
 's',
 'c',
 'u',
 's',
 's',
 'e',
 'd',
 '',
 '',
 '',
 'f',
 'o',
 'r',
 '',
 't',
 'h',
 'e',
 's',
 'e',
 '',
 'd',
 '',
 't',
 '',
 '',
 '',
 '',
 't',
 'h',
 'e',
 '',
 'c',
 'o',
 'r',
 'r',
 'e',
 'l',
 '',
 't',
 '',
 'v',
 'e',
 '',
 '',
 'n',
 '',
 'l',
 'y',
 's',
 '',
 's',
 '',
 '',
 'n',
 'd',
 '',
 'c',
 '',
 't',
 'e',
 's',
 '',
 'n',
 'e',
 'g',
 '',
 't',
 '

На мой взгляд, как-то долго для одного только абстракта. 

Попробую почистить через регулярные выражения значения \n, $ ,@xmath, @xcite

In [10]:
textt=' '
for x in test_token:
    textt += test_token[x]
textt

' for about 20 years the problem of properties of short - term changes of solar activity has been considered extensively .\nmany investigators studied the short - term periodicities of the various indices of solar activity .\nseveral periodicities were detected , but the periodicities about 155 days and from the interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often .\nfirst of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) .\nthis periodicity was confirmed for other solar flares data and for the same time period @xcite .\nit was also found in proton flares during solar cycles 19 and 20 @xcite , but it was not found in the solar flares data during solar cycles 22 @xcite .\n_    several autors confirmed above results for the daily sunspot area data . @xcite studied the sunspot data from 18741984 .\nshe found the 155-day periodicity in data records from 31 y

In [11]:
import re

In [12]:
# Прошу прощения за костыли в виде четырех строк ,но не было времени разобраться почему не работает map с такой же логикой

patterns = [r'\n' ,r'\@xmath', r'\$', r'\@xcite']
test_clear_article = test_token['article']
# test_clear_article = list(map(lambda x: ''.join(
#                               re.sub(x,'', test_clear_article)
# ), patterns))



test_clear_article = re.sub(r'\n','', test_clear_article)
test_clear_article = re.sub(r'@xmath*','', test_clear_article)
test_clear_article = re.sub(r'\$','', test_clear_article)
test_clear_article = re.sub(r'@xcite*','', test_clear_article)
test_clear_article

'for about 20 years the problem of properties of short - term changes of solar activity has been considered extensively .many investigators studied the short - term periodicities of the various indices of solar activity .several periodicities were detected , but the periodicities about 155 days and from the interval of 3  ] days ( 4  ] years ) are mentioned most often .first of them was discovered by  in the occurence rate of gamma - ray flares detected by the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) .this periodicity was confirmed for other solar flares data and for the same time period  .it was also found in proton flares during solar cycles 19 and 20  , but it was not found in the solar flares data during solar cycles 22  ._    several autors confirmed above results for the daily sunspot area data .  studied the sunspot data from 18741984 .she found the 155-day periodicity in data records from 31 years .this periodicity is always characteristic for one of 

In [13]:
def clear_text(text):
    test_clear_article = text
    test_clear_article = re.sub(r'\n','', test_clear_article)
    test_clear_article = re.sub(r'@xmath*','', test_clear_article)
    test_clear_article = re.sub(r'\$','', test_clear_article)
    test_clear_article = re.sub(r'@xcite*','', test_clear_article)
    return test_clear_article

In [29]:
y = next(test_iter)
for x in y:
    print(y[x])

for the hybrid monte carlo algorithm ( hmc)@xcite , often used to study quantum chromodynamics ( qcd ) on the lattice , one is interested in efficient numerical time integration schemes which are optimal in terms of computational costs per trajectory for a given acceptance rate . high order
numerical methods allow the use of larger step sizes , but demand a larger computational effort per step ; low order schemes do not require such large computational costs per step , but need more steps per trajectory .
so there is a need to balance these opposing effects .
omelyan integration schemes @xcite of a force - gradient type have proved to be an efficient choice , since it is easy to obtain higher order schemes that demand a small additional computational effort .
these schemes use higher - order information from force - gradient terms to both increase the convergence of the method and decrease the size of the leading error coefficient . other ideas to achieve better efficiency for numerica

In [24]:
# Проверяю работает ли все
text_test = clear_text(test_token['article'])
text_test

TypeError: expected string or bytes-like object, got 'dict'

In [15]:
tokenizer = get_tokenizer('basic_english') 

In [17]:
res_test_token = tokenizer(str(test_clear_article))
res_test_token

['for',
 'about',
 '20',
 'years',
 'the',
 'problem',
 'of',
 'properties',
 'of',
 'short',
 '-',
 'term',
 'changes',
 'of',
 'solar',
 'activity',
 'has',
 'been',
 'considered',
 'extensively',
 '.',
 'many',
 'investigators',
 'studied',
 'the',
 'short',
 '-',
 'term',
 'periodicities',
 'of',
 'the',
 'various',
 'indices',
 'of',
 'solar',
 'activity',
 '.',
 'several',
 'periodicities',
 'were',
 'detected',
 ',',
 'but',
 'the',
 'periodicities',
 'about',
 '155',
 'days',
 'and',
 'from',
 'the',
 'interval',
 'of',
 '3',
 ']',
 'days',
 '(',
 '4',
 ']',
 'years',
 ')',
 'are',
 'mentioned',
 'most',
 'often',
 '.',
 'first',
 'of',
 'them',
 'was',
 'discovered',
 'by',
 'in',
 'the',
 'occurence',
 'rate',
 'of',
 'gamma',
 '-',
 'ray',
 'flares',
 'detected',
 'by',
 'the',
 'gamma',
 '-',
 'ray',
 'spectrometer',
 'aboard',
 'the',
 '_',
 'solar',
 'maximum',
 'mission',
 '(',
 'smm',
 ')',
 '.',
 'this',
 'periodicity',
 'was',
 'confirmed',
 'for',
 'other',
 'solar',

Соединяю два текста (статья и абстракт) в один, чищу его и после посылю в токенизатор. 
UPD. Была проблема с тем, что на вход приходил генератор, но к ним нельзя обращаться по индексу, поэтому я приписал 
 y = next(data_iter) которые выводит dict с ключами. Возможно это ошибка, но это лучшее что я придумал в 23 часа. 

In [30]:
def yield_tokens(data_iter):
    y = next(data_iter)
    text = y['article'] +' '+ y['abstract']
    clear_article_text = clear_text(text)
    yield tokenizer(clear_article_text)

In [18]:
train_iter = iter(dataset['train'])

In [31]:
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [41]:
vocab(['not', 'in', 'for', 'green', 'monte'])

[305, 8, 59, 0, 745]